In [ ]:
!pip install ratsnlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 315 kB/s 
     |████████████████████████████████| 2.8 MB 9.6 MB/s 
     |████████████████████████████████| 57 kB 3.6 MB/s 
     |████████████████████████████████| 582 kB 44.4 MB/s 
     |████████████████████████████████| 512 kB 68.2 MB/s 
     |████████████████████████████████| 182 kB 33.8 MB/s 
     |████████████████████████████████| 880 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2683c1e804704ace9cf02e58d7310cf61bbaad0216a2f67af5b950ad547e851f
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/content/drive/MyDrive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-generation/epoch=0-val_loss=0.89.ckpt


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
if args.downstream_model_checkpoint_fpath is None:
    model = GPT2LMHeadModel.from_pretrained(
        args.pretrained_model_name,
    )
else:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)    
    pretrained_model_config = GPT2Config.from_pretrained(
        args.pretrained_model_name,
    )
    model = GPT2LMHeadModel(pretrained_model_config)
    fine_tuned_model_ckpt = torch.load(
        args.downstream_model_checkpoint_fpath,
        map_location=torch.device("cpu"),
    )
    model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Mounted at /content/drive


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
prompt="느린마을 막걸리 "
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 톡쏘는 탄산이 없고 부드럽고 진한맛이에요 단 맛이 강하지 않아서 좋네요

